In [1]:
import soup
from common import DotDict, JsonDict, SGR, StartDaemonThread
from utf8logger import CRITICAL, DEBUG, ERROR, INFO, PRINT, WARNING
import json, os, requests, time, traceback

class RequestError(Exception):
    pass

In [ ]:
class OneBotApi():
    def __init__(self, token=None, host='localhost', hp=5700, wp=5800, **kwargs) -> None:
        self.started = False
        self.host = host
        self.hp = hp
        self.wp = wp
        self.kwargs = kwargs
        self.headers = {'access_token':token,'ticket':token,'Authorization':token}

    ErrorCode = None
    ErrorTime = None

    def basicsession(self, mode, url, **kwargs):
        kw = kwargs
        kw.update(self.kwargs)
        while True:
            if not self.started:continue
            try:
                print(f'url:http://{self.host}:{self.hp}/{url} kw:{kw}')
                r = getattr(requests, mode)(f'http://{self.host}:{self.hp}/{url}',headers=self.headers , **kw)
                if r:r = DotDict(r.json())
                else:r = DotDict({'status': 'ok', 'retcode': 0, 'data': None, 'echo': ''})
                break
            except requests.exceptions.ConnectionError:
                ERROR('无法连接倒OneBot，请检查服务、地址、端口。')
            except Exception as e:
                raise RequestError(e)
        if (hasattr(r, 'status') and r.status != 'ok') or (hasattr(r,'retcode') and r.retcode != 0):
            ERROR(f'status: {r.status}, retcode:{r.retcode}, Msg: {r.msg}, Wording: {r.wording}')
            raise RequestError(f'status: {r.status}, retcode:{r.retcode}, Msg: {r.msg}, Wording: {r.wording}')
        return r.data

### 消息与事件上报 ###
    def pollForever(self, MessageAnalyst): # WS Adapter
        self.MessageAnalyst = MessageAnalyst
        import websocket
        while True:
            try:
                self.ws = websocket.create_connection(f'ws://{self.host}:{self.wp}', header=self.headers)
                recv = self.ws.recv()
                if not recv:raise
            except:
                ERROR('qsession.Poll 方法出错请检查连接配置', exc_info=True)
                time.sleep(15)
                continue
            recv = DotDict(recv)
            self.qq = recv.self_id
            self.started = True
            INFO(f'qq:{recv.self_id} 链接成功')
            while self.started:
                try:
                    recv = DotDict(self.ws.recv())
                    if recv.post_type == 'meta_event':continue
                    self.MessageAnalyst(recv)
                except Exception as e:
                    ERROR(e)
                    self.started = False
                    break

    def LoginInfo(self):
        '该接口用于获取 QQ 用户的登录号信息。'
        return self.basicsession('get','get_login_info')

    def SetProfile(self,nickname:str,company:str,email:str,college:str,personal_note:str,age:int=None,birthday:str=None):
        '''该接口用于设置 QQ 用户的个人资料信息。
    nickname		string	是	昵称
    company			string	是	公司
    email			string	是	邮箱
    college			string	是	大学
    personal_note	string	是	个人备注
    age				int32	否	年龄
    birthday		string	否	生日（格式：YYYY-MM-DD）'''
        payload = DotDict()
        payload.nickname = nickname
        payload.company = company
        payload.email = email
        payload.college = college
        payload.personal_note = personal_note
        if age:payload.age = age
        if birthday:payload.birthday = birthday
        return self.basicsession('post','set_qq_profile',json=payload)

    def List(self, mode:str, gid:int=None):
        '''该接口用于获取联系人列表。
    mode	string	是	获取对象类型：unfriend|friend|group|member
	gid		int		否	群ID'''
        mode = mode.lower()
        payload = DotDict()
        if mode == 'unfriend':url = 'get_unidirectional_friend_list'
        elif mode == 'friend':url = 'get_friend_list'
        elif mode == 'group':url = 'get_group_list'
        elif mode == 'member':
            url = 'get_group_member_list'
            payload.group_id = int(gid)
        return self.basicsession('post',url,json=payload)

    def Update(self, type:str=None):
        if type in ['friend',None]:
            self.FriendList = self.List('friend')
            self.Friend = lambda uid:[f for f in self.FriendList if f.user_id==uid][0]
        if type in ['group',None]:
            self.GroupList = self.List('group')
            self.Group = lambda gid:[g for g in self.GroupList if g.group_id==gid][0]
        if type in ['member',None]:
            self.MemberList = [m for g in self.GroupList for m in self.List('member',g.group_id)]
            self.Member = lambda gid,uid:[u for u in self.MemberList if u.group_id==gid and u.user_id==uid][0]

    def Info(self, mode:str, target:int, gid:int=None):
        '''获取联系人信息。
    mode	string	是	获取对象类型：unfriend|group|honor|member
	target	int		是	获取目标ID
	gid		int		否	群ID'''
        mode = mode.lower()
        payload = DotDict()
        if mode == 'unfriend':
            url = 'get_stranger_info'
            payload.user_id = int(target)
        elif mode == 'group':
            url = 'get_group_info'
            payload.group_id = int(target)
        elif mode == 'honor':
            url = 'get_group_honor_info'
            payload.group_id = int(target)
        elif mode == 'member':
            url = 'get_group_member_info'
            payload.user_id = int(target)
            payload.group_id = int(gid)
        return self.basicsession('post', url, json=payload)

    def SystemMsg(self, mode:str):
        '''该接口用于获取请求系统消息。
    mode	string	是	请求类型：friend|group'''
        return self.basicsession('get',f'get_{mode}_system_msg')

    def HistoryMsg(self, mode:str, target:int, start:int=0, count:int=0):
        '''获取历史消息
    id	int	是	消息ID'''
        mode = mode.lower()
        payload = DotDict()
        if mode == 'friend':
            payload.message_type = 'private'
            payload.user_id = target
        else:
            payload.message_type = 'group'
            payload.group_id = target
        if start:payload.message_seq = start
        if count:payload.count = count
        return self.basicsession('post','get_history_msg',json=payload)

    def GetMsg(self, id:int):
        '''获取消息
    id	int	是	消息ID'''
        return self.basicsession('post','get_msg',json={'message_id':id})

    def GetForward(self, id):
        '''获取合并转发内容
    id	string	是	消息资源ID（卡片消息里面的resId）'''
        return self.basicsession('post','get_forward_msg',json={'id':id})

    def SendMsg(self, mode:str, target:int, *message:object|list, reply:int=0, recall:int=0):
        '''该接口用于发送消息。
    '''
        mode = mode.lower()
        payload = DotDict()
        if mode == 'friend':
            payload.message_type = 'private'
            payload.user_id = target
        else:
            payload.message_type = 'group'
            payload.group_id = target
        if recall:payload.recall_duration = recall
        payload.message = list(message)+[soup.Reply(reply)] if reply else list(message)
        for n in range(len(payload.message)):
            payload.message[n] = {'type':message[n].pop('type'),'data':payload.message[n]}
            if 'data_type' in payload.message[n]['data']:payload.message[n]['data']['type'] = payload.message[n]['data'].pop('data_type')
        data = self.basicsession('post','send_msg',json=payload)
        if mode=="friend":INFO(f'发到好友{SGR(self.Friend(target),b4=11)}[{SGR(self.Friend(target).user_remark,b4=11)}({SGR(self.Friend(target).user_id,b4=1)})]{(reply and "回复("+SGR(reply,b4=2)+")") or ""}:\n{str(message)}')
        elif mode=="group":INFO(f'发到群{SGR(self.Group(target).group_name,b4=14)}({SGR(self.Group(target).group_id,b4=4)}){(reply and "回复("+SGR(reply,b4=2)+")") or ""}:\n{str(message)}')
        return data

    def SendForward(self, mode:str, target:int, *node_list:list):
        mode = mode.lower()
        if mode == 'friend':url = 'send_private_forward_msg'
        else:url = 'send_group_forward_msg'
        for node in node_list:
            if 'content' in node:
                node.content = [{'type':msg.pop('type'),'data':msg}for msg in node.content]
                node.data = {'content':node.pop('content')}
            else:node.data = {'id':node.pop('id')}
        return self.basicsession('post',url,json={'user_id' if mode == 'friend' else 'group_id':target,'messages':node_list})

    def Recall(self, id:int):
        '''该接口用于撤回消息。
    id	int	是	消息ID'''
        return self.basicsession('post','delete_msg',json={'message_id':id})

    def GetImage(self, file:str):
        return self.basicsession('post','get_image',json={'file':file})

    def GetRecord(self, file:str, out:str='mp3'):
        return self.basicsession('post','get_record',json={'file':file,'out_format':out})

    def GetFile(self, file:str, type:str='base64'):
        return self.basicsession('post','get_file',json={'file':file,'file_type':type})

    def request_response(self, request:dict, approve:bool=False, re:str=None):
        payload = DotDict()
        payload.flag = request.flag
        payload.approve = approve
        if request.request_type == "friend":
            if re:payload.remark = re
        else:
            payload.sub_type = request.sub_type
            if re:payload.reason = re
        print(payload)
        return self.basicsession('post',f'set_{"friend" if request.request_type == "friend" else "group"}_add_request', json=payload)

    def SetGroupName(self, gid:int, name:str):
        return self.basicsession('post','set_group_name', json={'group_id':gid, 'group_name':name})

    def SetGroupAdmin(self, gid:int, uid:int, enable:True):
        return self.basicsession('post','set_group_admin', json={'group_id':gid, 'user_id':uid, 'enable':enable})

    def SetGroupCard(self, gid:int, uid:int, card:str=None):
        payload = DotDict({'group_id':gid, 'user_id':uid})
        if card:payload.card = card
        return self.basicsession('post','set_group_admin', json=payload)
'''
get_essence_msg_list #获取精华消息列表
clear_msgs #清除本地缓存消息
is_blacklist_uin #QQ是否在黑名单内
send_like #点赞资料卡
'''

'\nget_essence_msg_list #获取精华消息列表\nclear_msgs #清除本地缓存消息\nis_blacklist_uin #QQ是否在黑名单内\nsend_like #点赞资料卡\n'

In [ ]:
host = '192.168.0.25'
hp = 5700
wp = 5800
token = '1064393873'
bot = OneBotApi('1064393873',host,hp,wp)
bot.started = True
bot.Update()

url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}
[24-06-13 13:03:46][ERROR][3329598455.basicsession] 无法连接倒OneBot，请检查服务、地址、端口。
url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}
[24-06-13 13:03:49][ERROR][3329598455.basicsession] 无法连接倒OneBot，请检查服务、地址、端口。
url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}
[24-06-13 13:03:51][ERROR][3329598455.basicsession] 无法连接倒OneBot，请检查服务、地址、端口。
url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}
[24-06-13 13:03:54][ERROR][3329598455.basicsession] 无法连接倒OneBot，请检查服务、地址、端口。
url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}
[24-06-13 13:03:56][ERROR][3329598455.basicsession] 无法连接倒OneBot，请检查服务、地址、端口。
url:http://192.168.0.25:5700/get_friend_list kw:{'json': {}}


In [7]:
StartDaemonThread(bot.pollForever,INFO)

[24-06-06 20:49:14][INFO][2130481560.pollForever] qq:3338059470 链接成功
[24-06-06 21:48:25][INFO][2130481560.pollForever] {'time': 1717681706, 'self_id': 3338059470, 'post_type': 'message', 'message_type': 'group', 'sub_type': 'normal', 'message_id': 947683572, 'group_id': 260715723, 'peer_id': 3338059470, 'user_id': 1064393873, 'message': [{'data': {'id': 2}, 'type': 'basketball'}], 'raw_message': '[CQ:basketball,id=2]', 'font': 0, 'sender': {'user_id': 1064393873, 'nickname': '受、死', 'card': '', 'role': 'owner', 'title': '', 'level': ''}}


In [37]:
bot.SendMsg('friend',1064393873,{'type': 'face','id':218})

url:http://192.168.0.25:5700/send_msg kw:{'json': {'message_type': 'private', 'user_id': 1064393873, 'message': [{'type': 'face', 'data': {'id': 218}}]}}
[24-06-11 17:39:13][INFO][3329598455.SendMsg] 发到好友受、死[Admin(1064393873)]:
({'id': 218},)


{'message_id': 1473034190, 'time': 1718098753}

In [24]:
bot.HistoryMsg('group',260715723)

http://192.168.0.25:5700/get_history_msg {'json': {'message_type': 'group', 'group_id': 260715723}}


{'status': 'ok',
 'retcode': 0,
 'data': {'messages': [{'time': 1717545678,
    'message_type': 'group',
    'message_id': 1769038595,
    'message_id_qq': 7376802517574084107,
    'message_seq': 32741,
    'real_id': 32741,
    'sender': {'user_id': 2907237958,
     'nickname': '精神病院里的人',
     'sex': 'unknown',
     'age': 0,
     'uid': 'u_XGLNBZyp3QKeaXiEqaWQjw',
     'tiny_id': 'u_XGLNBZyp3QKeaXiEqaWQjw'},
    'message': [{'type': 'forward',
      'data': {'summary': '查看20条转发消息',
       'filename': '1717545677',
       'id': 'Dkcd7MWfnvaC9Vy/N48PpRAt4qp+Gjwqnduabj7+Ng69ieo5zClecixC5HwXjTmc',
       'desc': 'QQ管家: 标题:空蛍 Pid:119258477\n作者:pottsness Uid:5933626\nQQ管家: 标题:雨を手繰る Pid:119258429\n作者:しぐれうい Uid:431873 F\nQQ管家: 标题:【創作】氷属性男子とクールな同僚女子50 Pid:119258837\n作者:殿ヶ谷\n...'}}],
    'group_id': 260715723,
    'peer_id': 260715723},
   {'time': 1717545691,
    'message_type': 'group',
    'message_id': 1870038143,
    'message_id_qq': 7376802573347961699,
    'message_seq': 32742,
    'rea

In [18]:
bot.SystemMsg('group')

url:http://192.168.0.25:5700/get_group_system_msg kw:{}


{'invited_requests': [],
 'join_requests': [{'request_id': 1717663758299903,
   'invitor_uin': 0,
   'invitor_nick': '',
   'group_id': 260715723,
   'group_name': '鸡兔同笼',
   'checked': False,
   'actor': 0,
   'requester_uin': 2640277945,
   'requester_nick': '萝卜TD',
   'message': '1\n',
   'flag': '1717663758299903;260715723;2640277945'}]}

In [9]:
bot.List('friend')

http://192.168.0.25:5700/get_friend_list {'json': {'group_id': None}}


{'status': 'ok',
 'retcode': 0,
 'data': [{'user_id': 2907237958,
   'user_name': '精神病院里的人',
   'user_displayname': '精神病院里的人',
   'user_remark': '精神病院里的人',
   'age': 0,
   'gender': 1,
   'group_id': 0,
   'platform': 'MOBILE_HD',
   'term_type': 65805},
  {'user_id': 66600000,
   'user_name': 'babyQ',
   'user_displayname': 'babyQ',
   'user_remark': 'babyQ',
   'age': 0,
   'gender': 1,
   'group_id': 0,
   'platform': 'MOBILE_OTHER',
   'term_type': 65794},
  {'user_id': 1064393873,
   'user_name': '受、死',
   'user_displayname': 'Admin',
   'user_remark': 'Admin',
   'age': 23,
   'gender': 1,
   'group_id': 0,
   'platform': 'MOBILE_ANDROID',
   'term_type': 65799},
  {'user_id': 3338059470,
   'user_name': '为人民服务',
   'user_displayname': '为人民服务',
   'user_remark': '为人民服务',
   'age': 0,
   'gender': 0,
   'group_id': 0,
   'platform': 'MOBILE_ANDROID',
   'term_type': 65799},
  {'user_id': 1721651613,
   'user_name': '凶猛猪王',
   'user_displayname': '凶猛猪王',
   'user_remark': '凶猛猪王',
 

In [8]:
bot.List('group')

http://192.168.0.25:5700/get_group_list {'json': {'group_id': None}}


{'status': 'ok',
 'retcode': 0,
 'data': [{'group_id': 260715723,
   'group_name': '鸡兔同笼',
   'group_remark': '',
   'group_uin': 4150715723,
   'admins': [1064393873, 2907237958, 3338059470],
   'class_text': '',
   'is_frozen': False,
   'max_member': 200,
   'member_num': 5,
   'member_count': 5,
   'max_member_count': 200}],
 'echo': ''}

In [15]:
bot.List('member',260715723)

http://192.168.0.25:5700/get_group_member_list {'json': {'group_id': 260715723}}


{'status': 'ok',
 'retcode': 0,
 'data': [{'user_id': 1064393873,
   'group_id': 260715723,
   'user_name': '受、死',
   'sex': 'female',
   'age': 0,
   'title': '',
   'title_expire_time': 0,
   'nickname': '受、死',
   'user_displayname': '',
   'card': '',
   'distance': 100,
   'honor': [],
   'join_time': 1669711183,
   'last_active_time': 1717734726,
   'last_sent_time': 1717734726,
   'unique_name': '',
   'area': '',
   'level': 10315,
   'role': 'owner',
   'unfriendly': False,
   'card_changeable': True,
   'shut_up_timestamp': 0},
  {'user_id': 2907237958,
   'group_id': 260715723,
   'user_name': '精神病院里的人',
   'sex': 'female',
   'age': 0,
   'title': '',
   'title_expire_time': 0,
   'nickname': '精神病院里的人',
   'user_displayname': '精神病院',
   'card': '精神病院',
   'distance': 100,
   'honor': [3],
   'join_time': 1681203484,
   'last_active_time': 1717774079,
   'last_sent_time': 1717774079,
   'unique_name': '',
   'area': '',
   'level': 10315,
   'role': 'admin',
   'unfriendly': 